NEW CODE HERE

In [1]:
# this is required since Juypyter notebook is not async by default.
import nest_asyncio
nest_asyncio.apply()

In [1]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("/teamspace/studios/this_studio/data").load_data()

In [2]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [4]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [5]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

vector_query_engine = vector_index.as_query_engine()

In [6]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [7]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    # allows us to view the intermediate steps of the query engine
    verbose=True,
)

In [10]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document provides detailed information on various tropical locations and states that offer production incentives for filming, focusing on both scripted and unscripted productions. It outlines the incentives available, such as tax credits, rebates, and minimum spend requirements, and discusses considerations like content qualification, application deadlines, qualifying expenses, and potential costs. The text highlights the benefits of utilizing incentives for unscripted productions to reduce costs, increase competitiveness, and promote tourism and economic growth. It also emphasizes the importance of film commissions targeting unscripted productions by aligning minimum spend requirements with budgets, offering alternative benefits, easing reporting requirements, and promoting locations directly to production companies to attract economic benefits and foster cultural exchange.


In [11]:
response = query_engine.query("How can production assistants best use this information?")
print(str(response))

Selecting query engine 1: Production assistants can use this information to retrieve specific context from the MetaGPT paper, which can help them better understand the content and make informed decisions..
Production assistants can best use this information by understanding that the application process for incentives can vary depending on the specific circumstances of the production. They should be aware that sometimes a local entity must apply, while in other cases, a production company or even the broadcaster/streamer/studio can apply. It is important for production assistants to be familiar with these variations and to ensure that the appropriate entity applies for the incentives to maximize the benefits for the project.


In [12]:
response = query_engine.query("What is the holy grail?")
print(str(response))

Selecting query engine 0: The question is asking about the usefulness of MetaGPT in summarization questions, which aligns with choice (1) being relevant to summarization questions related to MetaGPT..
The holy grail is the ultimate goal or ideal combination of incentives, such as tax credits, rebates, and bonuses, that can significantly reduce production costs and maximize savings for film and TV projects.


In [13]:
response = query_engine.query("Tell me everything you know about transferable tax credit")
print(str(response))

Selecting query engine 1: The question is asking for specific information about transferable tax credit, which is more related to retrieving specific context from the MetaGPT paper..
Transferable tax credits can be applied against the production entity's tax liability in a specific jurisdiction or sold on the open market if the entity has no tax liability. There is usually a market for these credits, allowing buyers to purchase them at a discount. Selling the credit can result in quicker cash access compared to refundable tax credits. It is important to consider the discount rate when planning to sell or transfer a tax credit to avoid unexpected outcomes. Transferable tax credits are offered in locations like the Dominican Republic and the US state of Georgia.
